In [1]:
import pandas as pd
import datetime
import time
import pprint as pp
import json
import numpy as np

In [2]:
df = pd.read_csv('./clipper-clean.csv')

In [3]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [4]:
df = df.sort_values(by=['datetime', 'position']).round(5)

In [11]:
df['status'].fillna("None")

0        racing
1        racing
2        racing
3        racing
4        racing
5        racing
6        racing
7        racing
8        racing
9        racing
10       racing
11       racing
12       racing
13       racing
14       racing
15       racing
16       racing
17       racing
18       racing
19       racing
20       racing
21       racing
22       racing
23       racing
24       racing
25       racing
26       racing
27       racing
28       racing
29       racing
          ...  
14573    racing
14574    racing
14575    racing
14576    racing
14577    racing
14578    racing
14579    racing
14580    racing
14581    racing
14582    racing
14583    racing
14584    racing
14585    racing
14586    racing
14587    racing
14588    racing
14589    racing
14590    racing
14591    racing
14592    racing
14593    racing
14594    racing
14595    racing
14596    racing
14597    racing
14598    racing
14599    racing
14600    racing
14601    racing
14602    racing
Name: status, Length: 14

In [14]:
df[df['status'] == np.nan]

,Unnamed: 0,position,team,lat,lon,dtf,dist_mg,status,datetime,race


In [7]:
races = []
for race in df['race'].unique():
    
    df_tmp = df[df['race'] == race]
    race_dict = {
        "race": race,
        "start_date": time.mktime(df_tmp['datetime'].min().timetuple()),
        "end_date": time.mktime(df_tmp['datetime'].max().timetuple()),
        "positions": []
    }
    
    positions = race_dict['positions']
    
    for i, dt in enumerate(df_tmp['datetime'].unique()):
        rows = df_tmp[df_tmp['datetime'] == dt]

        lon_sum = rows['lon'].sum()
        lat_sum = rows['lat'].sum()
        num_ships = len(rows)
        centroid = [lon_sum / num_ships, lat_sum / num_ships]
        bbox = [[rows['lon'].min(), rows['lat'].min()], \
                [rows['lon'].max(), rows['lat'].max()]]

        timestamp = time.mktime(rows.iloc[0,]['datetime'].timetuple())

        positions_at_time = []

        for ship in rows.iterrows():
            d = ship[1]
            team = d['team'].lower().replace(' ', '-')
            position = d['position']
            coords = [d['lon'], d['lat']]
            lon_sum += d['lon']
            lat_sum += d['lat']
            dtf = d['dtf']
            dist_mg = d['dist_mg']
            status = d['status'] if d['status'] != np.nan else "None"

            ship_data = {
                "team": team,
                "position": position,
                "coords": coords,
                "dtf": dtf, 
                "dist_mg": dist_mg,
                "status": status

            }
            positions_at_time.append(ship_data)


        fleet_snapshot = {
            "time": timestamp,
            "fleet_bbox": bbox, 
            "fleet_centroid": centroid,
            "ship_data": positions_at_time
        }

        positions.append(fleet_snapshot)

    races.append(race_dict)
    

In [8]:
# pp.pprint(race_dict)
with open('./clipper-clean-.json', 'w') as out:
    json.dump(races, out)

In [15]:
max_dt_wna = df[df['race'] == 'west-north-america']['datetime'].max()

In [17]:
df['race'].unique()

array(['pacific', 'west-north-america', 'east-north-america',
       'north-atlantic', 'british-isles'], dtype=object)

In [16]:
df[df['datetime'] == max_dt_wna]

,Unnamed: 0,position,team,lat,lon,dtf,dist_mg,status,datetime,race
6960,6965,1,Nasdaq,10.17007,-79.48637,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6961,6966,2,PSP Logistics,10.17130,-79.48127,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6962,6967,3,Unicef,10.17007,-79.48867,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6963,6968,4,Garmin,10.17080,-79.48244,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6964,6969,5,Hotelplanner.com,10.17054,-79.47758,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6965,6970,6,Liverpool 2018,10.17022,-79.48761,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6966,6971,7,Visit Seattle,10.17050,-79.48446,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6967,6972,8,Qingdao,10.17096,-79.48343,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6968,6973,9,GREAT Britain,10.17131,-79.47978,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
6969,6974,10,Dare To Lead,10.17055,-79.47822,0.0,55,NaN,2018-06-03 22:00:00,west-north-america
